# Core Graph Elements
**Objectives:**  
- Understand State, Node, Edge, Start/End points  
- Learn Tools & ToolNodes  
- Build and compile a simple StateGraph  
- Review message types (Human, AI, System, Tool, Function)

%pip install langchain --quiet

from typing import TypedDict
from langraph import StateGraph, ToolNode
from langchain.schema import HumanMessage, AIMessage, SystemMessage, ToolMessage
## State
A `State` is a shared data structure for carrying context through nodes.

In [ ]:
class AgentState(TypedDict):
    counter: int

state: AgentState = { 'counter': 0 }
print(state)

## Node
A Node is a Python function that takes and returns the state.

In [ ]:
def increment_node(state: AgentState) -> AgentState:
    """Increment the counter by 1."""
    state['counter'] += 1
    return state

# Test:
state = increment_node(state)
print(state)

## Edge
Connect nodes by directing state flow: `graph.add_edge(from, to)`.

In [ ]:
from langraph import Edge # (will be used when building the graph below)

## Start and End Points
Define where execution begins and finishes.

In [ ]:
graph = StateGraph(AgentState)
graph.set_entry_point('start')
graph.set_finish_point('end')

## Tools & ToolNode
Tools are external utilities; wrap them in a ToolNode.

In [ ]:
def sample_tool(x: int) -> int:
    """Example tool: squares its input."""
    return x * x

tool_node = ToolNode(name='square_tool', tool=sample_tool)

## StateGraph & Runnable
Build, compile, and invoke your graph.

In [ ]:
# Add nodes and edges
graph.add_node('increment', increment_node)
graph.add_node('square', tool_node)
graph.add_edge('start', 'increment')
graph.add_edge('increment', 'square')
graph.add_edge('square', 'end')

# Compile and run
app = graph.compile()
result = app.invoke(counter=0)
print('Final state:', result)

## Message Types
- `HumanMessage`: user input  

In [ ]:
print(HumanMessage(content='Hello'))
print(AIMessage(content='Hi there!'))
print(SystemMessage(content='You are an assistant.'))
print(ToolMessage(content='42', tool_call='answer'))

## Visualize Graph
Render the node connections.

In [ ]:
from graphviz import Digraph

dot = Digraph()
dot.node('start'); dot.node('increment'); dot.node('square'); dot.node('end')
dot.edges(['startincrement', 'incrementsquare', 'squareend'])
dot

### Exercise
Build a two-node graph that takes `counter` from 0 to 2:
1. Node A: +1  
2. Node B: +1  
Compile and invoke to verify `counter == 2`.

In [ ]:
# TODO: define nodes A and B, add edges, compile & invoke

In [ ]:
def node_a(s): s['counter'] += 1; return s
def node_b(s): s['counter'] += 1; return s
g = StateGraph(AgentState)
g.set_entry_point('A'); g.add_node('A', node_a)
g.add_edge('A', 'B'); g.add_node('B', node_b)
g.set_finish_point('B')
app = g.compile()
print(app.invoke(counter=0))  # {'counter': 2}